In [16]:
import numpy as np
import torch
import torch.nn.functional as F
import math
from DRQN.network import ADRQN
from DRQN.experience_replay import ExpBuffer
from DRQN.agent import Agent
from DRQN.epsilon_greedy_strategy import EpsilonGreedyStrategy
from DRQN.plot import plot, get_moving_average
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
%run ./env.ipynb

In [17]:
import pandas as pd
df = pd.DataFrame({'iteration': 0,"epsiode_num":0, "reward":0}, index=[0])



In [18]:
def save(model, path):
    torch.save(model.state_dict(), path)    

def load(path,n_actions,state_size, embedding_size):
    model = ADRQN(n_actions,state_size,embedding_size)
    model.load_state_dict(torch.load(path))
    return model   


In [19]:
env = EnvManager(device)
state_size = env.num_state_features()
n_actions = env.num_actions_available()


In [20]:
embedding_size = 16
M_episodes = 2000
memory_size = 1000
sample_length = 100
replay_buffer = ExpBuffer(memory_size, sample_length)
batch_size = 8
eps_start = 0.9
eps = eps_start
eps_end = 0.05
eps_decay = 10
gamma = 0.999
learning_rate = 0.001
blind_prob = 0
EXPLORE = 1000
target_update = 100  # Interval for updating the target net

In [21]:
strategy = EpsilonGreedyStrategy(eps_start, eps_end, eps_decay)
agent = Agent(strategy, n_actions, device)
adrqn = ADRQN(n_actions, state_size, embedding_size).cuda()
adrqn_target = ADRQN(n_actions, state_size, embedding_size).cuda()
adrqn_target.load_state_dict(adrqn.state_dict())
optimizer = torch.optim.Adam(adrqn.parameters(), lr=learning_rate)

In [22]:
# Load the model
model = load("./model/model",n_actions, state_size, embedding_size)


In [26]:
rightClassified = 0
newDF = pd.DataFrame({"epsiode_num":0, "reward":0}, index=[0])

for j in range(113):
    env.reset(j)
    last_observation = env.get_state()
    episode_duration = 0
    last_action = 0
    current_return = 0
    done = False
    while not done:
            action, hidden = agent.act(torch.tensor(last_observation).float().view(1, 1, -1).cuda(), F.one_hot(
                torch.tensor(last_action), n_actions).view(1, 1, -1).float().cuda(), eps, adrqn, hidden=hidden)

            observation, reward, done  = env.take_action(action)
            reward = np.sign(reward)
            current_return += reward
            episode_duration =  episode_duration +1
            last_action = action
            last_observation = observation 
            if done:
                break

    acc = current_return / episode_duration
    new_row = pd.DataFrame({"epsiode_num": j ,"reward":acc}, index=[0])
    newDF = pd.concat([newDF,new_row],ignore_index=True)

    if (acc >= 0.8):
        rightClassified = rightClassified + 1
    else:
        print(env.get_episode_name())


print(rightClassified)

['Special methods should have an expected number of parameters']
['Special methods should have an expected number of parameters']
['Instance and class methods should have at least one positional parameter']
['Instance and class methods should have at least one positional parameter']
['Boolean expressions of exceptions should not be used in "except" statements']
['Boolean expressions of exceptions should not be used in "except" statements']
['Item operations should be done on objects supporting them']
['Item operations should be done on objects supporting them']
['Raised Exceptions must derive from BaseException']
['Raised Exceptions must derive from BaseException']
['Custom Exception classes should inherit from "Exception" or one of its subclasses']
['Custom Exception classes should inherit from "Exception" or one of its subclasses']
['"=+" should not be used instead of "+="']
['"=+" should not be used instead of "+="']
['All code should be reachable']
['All code should be reachable']


In [27]:
df

,iteration,epsiode_num,reward
0,0,0,0


In [23]:
df["iteration"].to_csv("data.csv",index=False)

In [28]:
newDF["reward"].to_csv("data1.csv",index=False)